In [32]:
import numpy as np
from datetime import datetime
import plotly
import plotly.graph_objs as go
import chart_studio.plotly as py 
import random
import math

LatMin =  70
LatMax =  90
LatRange = LatMax - LatMin

# Construct random functions of the type:
# f(LAT,LON) = SpikeHeight * math.e ^ ( - ((LAT+slideLat)^2) / (2*sigmaLat^2)  -  ((LON+slideLon)^2) / (2*sigmaLon^2) )
Deltas = list()
for i in range( 0, 200 ):
    SpikeHeight = 0.01 * (random.random()-0.5) # spike height
    sigmaLat    = random.random() * 20         # spike width at Latitudes
    slideLat    = random.random() * 180 - 90   # spike position at Latitudes
    sigmaLon    = random.random() * 20         # spike width at Longitudes
    slideLon    = random.random() * 360 - 180  # spike position at Longitudes
    Deltas.append( [SpikeHeight, sigmaLat, slideLat, sigmaLon, slideLon] )
     
# init
Lons = list( range( -180, 180, 2 ) )
Lats = list( range(  -90,  90, 2 ) )
Zaxis = np.zeros( (len(Lons), len(Lats)) ).tolist()
Data  = np.zeros( (len(Lons), len(Lats)) ).tolist()

# Apply functions at the whole lat-lon grid
for i in range( 0, len(Lons) ):
    for j in range( 0, len(Lats) ):
        LON = Lons[i]
        LAT = Lats[j]
        for D in Deltas:
            SpikeHeight = D[0]
            sigmaLat    = D[1]
            slideLat    = D[2]
            sigmaLon    = D[3]
            slideLon    = D[4]
            
            sigma_mod =  (LatMax-LatMin)/2.3548
            LatitudeModulation = math.e**( -((LAT-LatMax/2-LatMin/2)**2) / (2*sigma_mod**2) )
            
            Data[i][j] +=  LatitudeModulation * SpikeHeight * math.e ** ( - ((LAT-slideLat)**2)/(2*sigmaLat**2) - ((LON    -slideLon)**2)/(2*sigmaLon**2)  )
            
            # respect the boundary conditions: spikes at the edge of Longitudes have to be applied to the other edge as well so that the grid becomes a cylinder
            Data[i][j] +=  LatitudeModulation * SpikeHeight * math.e ** ( - ((LAT-slideLat)**2)/(2*sigmaLat**2) - ((LON-360-slideLon)**2)/(2*sigmaLon**2)  )
            Data[i][j] +=  LatitudeModulation * SpikeHeight * math.e ** ( - ((LAT-slideLat)**2)/(2*sigmaLat**2) - ((LON+360-slideLon)**2)/(2*sigmaLon**2)  )
        

        
        
# define the layout of the plot
theLayout = dict( title="Sub Grid Variability", width=1000, height=800,
        scene = dict(
            xaxis = dict(title="Latitude",dtick=10), yaxis = dict(title="Longitude",dtick=20), zaxis = dict( title="" ),
        )
)



surface=go.Surface(x=Lats, y=Lons, z=Data,   colorscale = "jet", surfacecolor = Data,
        colorbar=dict(thickness=20, len=0.75, ticklen=4, title="Value"),
)
Plotables = list()
Plotables.append( surface )
fig = dict( data=Plotables, layout=theLayout )
plotly.offline.init_notebook_mode(connected=True)
plotly.offline.iplot(fig)

surface=go.Surface(x=Lats, y=Lons, z=Zaxis,   colorscale = "jet", surfacecolor = Data,
        colorbar=dict(thickness=20, len=0.75, ticklen=4, title="Value"),
)
Plotables = list()
Plotables.append( surface )
fig = dict( data=Plotables, layout=theLayout )
plotly.offline.init_notebook_mode(connected=True)
plotly.offline.iplot(fig)


    